In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

sns.set(rc={"figure.figsize": (10, 6)})
sns.set_style("darkgrid")

import os
import sys
module_path = os.path.abspath(os.path.join('.'))
sys.path.append(module_path)

from utils import *
from plot import *
from model import SpatialModel

In [2]:
df = pd.read_csv('oto_points.csv')
df.head()

,price,rooms,surface,address,lat,lng,store,store_closest,school,school_closest,...,cafe,cafe_closest,gym,gym_closest,church,church_closest,train_station,train_station_closest,transit_station,transit_station_closest
0,610000.0,2,63.58,"pl. św. Macieja, Nadodrze, Śródmieście, Wrocła...",51.119903,17.035325,15,13.916640,12,85.439551,...,19,44.081468,13,116.215879,17,167.171816,2,516.682872,16,43.091362
1,330000.0,1,28.63,"pl. św. Macieja, Nadodrze, Śródmieście, Wrocła...",51.119903,17.035325,15,13.916640,12,85.439551,...,19,44.081468,13,116.215879,17,167.171816,2,516.682872,16,43.091362
2,450000.0,1,30.00,"pl. św. Macieja, Nadodrze, Śródmieście, Wrocła...",51.119903,17.035325,15,13.916640,12,85.439551,...,19,44.081468,13,116.215879,17,167.171816,2,516.682872,16,43.091362
3,755000.0,1,47.42,"pl. św. Macieja, Nadodrze, Śródmieście, Wrocła...",51.119903,17.035325,15,13.916640,12,85.439551,...,19,44.081468,13,116.215879,17,167.171816,2,516.682872,16,43.091362
4,499000.0,2,34.30,"Popowice Południowe, Fabryczna, Wrocław, dolno...",51.126858,16.986331,15,58.760925,13,174.189024,...,19,222.701686,15,180.152352,9,133.983686,0,NaN,14,248.316037


In [3]:
coords = valid_address(df)

In [4]:
coords['log_price'] = np.log10(coords['price'])

In [5]:
quartiles = coords.groupby('valid_address')['log_price'].quantile(q=[0., .5, 1.]).reset_index(-1).rename(columns={'level_1': 'q'})

In [6]:
plot_df = pd.merge(quartiles, coords.groupby('valid_address').size().rename('size'), left_index=True, right_index=True)
plot_df['radius'] = np.sqrt(plot_df['size'] + 1) #* (plot_df['q'] + 0.5) / 1.5
plot_df = pd.merge(plot_df, coords[['lat', 'lng', 'valid_address']].drop_duplicates('valid_address'), left_index=True, right_on='valid_address')

In [7]:
plot_map(coords)